In [2]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change



In [3]:
trainfile = r'C:\Users\farha\Downloads\Insurance Fraud - TRAIN-3000.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'C:\Users\farha\Downloads\Insurance Fraud -TEST-12900.csv'
testData = pd.read_csv(testfile)  #creates a dataframe


print(trainData.shape)
print(testData.shape)



(2999, 32)
(12918, 32)


In [4]:
TrainCols = list(trainData.columns.values)
TestCols = list(testData.columns.values)
print(TrainCols)
print(TestCols)

['MONTH', 'WEEKOFMONTH', 'DAYOFWEEK', 'MAKE', 'ACCIDENTAREA', 'DAYOFWEEKCLAIMED', 'MONTHCLAIMED', 'WEEKOFMONTHCLAIMED', 'SEX', 'MARITALSTATUS', 'AGE', 'FAULT', 'POLICYTYPE', 'VEHICLECATEGORY', 'VEHICLEPRICE', 'REPNUMBER', 'DEDUCTIBLE', 'DRIVERRATING', 'DAYS_POLICY_ACCIDENT', 'DAYS_POLICY_CLAIM', 'PASTNUMBEROFCLAIMS', 'AGEOFVEHICLE', 'AGEOFPOLICYHOLDER', 'POLICEREPORTFILED', 'WITNESSPRESENT', 'AGENTTYPE', 'NUMBEROFSUPPLIMENTS', 'ADDRESSCHANGE_CLAIM', 'NUMBEROFCARS', 'YEAR', 'BASEPOLICY', 'FRAUDFOUND']
['MONTH', 'WEEKOFMONTH', 'DAYOFWEEK', 'MAKE', 'ACCIDENTAREA', 'DAYOFWEEKCLAIMED', 'MONTHCLAIMED', 'WEEKOFMONTHCLAIMED', 'SEX', 'MARITALSTATUS', 'AGE', 'FAULT', 'POLICYTYPE', 'VEHICLECATEGORY', 'VEHICLEPRICE', 'REPNUMBER', 'DEDUCTIBLE', 'DRIVERRATING', 'DAYS_POLICY_ACCIDENT', 'DAYS_POLICY_CLAIM', 'PASTNUMBEROFCLAIMS', 'AGEOFVEHICLE', 'AGEOFPOLICYHOLDER', 'POLICEREPORTFILED', 'WITNESSPRESENT', 'AGENTTYPE', 'NUMBEROFSUPPLIMENTS', 'ADDRESSCHANGE_CLAIM', 'NUMBEROFCARS', 'YEAR', 'BASEPOLICY', 'F

In [5]:
# Seperate Target column from Train Data
Xtrain = trainData[TrainCols[0:len(TrainCols)-1]]
Ytrain = trainData[['FRAUDFOUND']]
print("Train Set shape:")
print(Xtrain.shape)
print(Ytrain.shape)
Xtest = testData[TestCols[0:len(TestCols)-1]]
Ytest = testData[['FRAUDFOUND']]
print("Test Set shape:")
print(Xtest.shape)
print(Ytest.shape)

Train Set shape:
(2999, 31)
(2999, 1)
Test Set shape:
(12918, 31)
(12918, 1)


In [6]:
Ytest

,FRAUDFOUND
0,Yes
1,Yes
2,Yes
3,Yes
4,Yes
...,...
12913,No
12914,No
12915,No
12916,No


In [7]:
from sklearn.preprocessing import OrdinalEncoder

# instantiate the preprocessing model ordinal encoder for test
oe = OrdinalEncoder()
Ytest['FRAUDFOUND_transformed_Test'] = oe.fit_transform(Ytest[['FRAUDFOUND']])
Ytest = Ytest.drop('FRAUDFOUND', axis=1)

In [8]:
Ytrain

,FRAUDFOUND
0,Yes
1,Yes
2,Yes
3,Yes
4,Yes
...,...
2994,No
2995,No
2996,No
2997,No


In [9]:
# instantiate the preprocessing model ordinal encoder for train data
Ytrain['FRAUDFOUND_transformed_Train'] = oe.fit_transform(Ytrain[['FRAUDFOUND']])
Ytrain = Ytrain.drop('FRAUDFOUND', axis=1)

In [10]:
Ytrain,Ytest

(      FRAUDFOUND_transformed_Train
 0                              1.0
 1                              1.0
 2                              1.0
 3                              1.0
 4                              1.0
 ...                            ...
 2994                           0.0
 2995                           0.0
 2996                           0.0
 2997                           0.0
 2998                           0.0
 
 [2999 rows x 1 columns],
        FRAUDFOUND_transformed_Test
 0                              1.0
 1                              1.0
 2                              1.0
 3                              1.0
 4                              1.0
 ...                            ...
 12913                          0.0
 12914                          0.0
 12915                          0.0
 12916                          0.0
 12917                          0.0
 
 [12918 rows x 1 columns])

In [11]:
col=Xtrain.columns.values
col

array(['MONTH', 'WEEKOFMONTH', 'DAYOFWEEK', 'MAKE', 'ACCIDENTAREA',
       'DAYOFWEEKCLAIMED', 'MONTHCLAIMED', 'WEEKOFMONTHCLAIMED', 'SEX',
       'MARITALSTATUS', 'AGE', 'FAULT', 'POLICYTYPE', 'VEHICLECATEGORY',
       'VEHICLEPRICE', 'REPNUMBER', 'DEDUCTIBLE', 'DRIVERRATING',
       'DAYS_POLICY_ACCIDENT', 'DAYS_POLICY_CLAIM', 'PASTNUMBEROFCLAIMS',
       'AGEOFVEHICLE', 'AGEOFPOLICYHOLDER', 'POLICEREPORTFILED',
       'WITNESSPRESENT', 'AGENTTYPE', 'NUMBEROFSUPPLIMENTS',
       'ADDRESSCHANGE_CLAIM', 'NUMBEROFCARS', 'YEAR', 'BASEPOLICY'],
      dtype=object)

In [12]:
categoricalFeatures = ['MONTH', 'DAYOFWEEK', 'MAKE', 'ACCIDENTAREA','DAYOFWEEKCLAIMED', 'MONTHCLAIMED', 'SEX','MARITALSTATUS', 'FAULT', 'POLICYTYPE', 'VEHICLECATEGORY','VEHICLEPRICE','DAYS_POLICY_ACCIDENT', 'DAYS_POLICY_CLAIM', 'PASTNUMBEROFCLAIMS','AGEOFVEHICLE', 'AGEOFPOLICYHOLDER', 'POLICEREPORTFILED','WITNESSPRESENT', 'AGENTTYPE', 'NUMBEROFSUPPLIMENTS','ADDRESSCHANGE_CLAIM', 'NUMBEROFCARS', 'BASEPOLICY']

In [13]:
Xtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   MONTH                 2999 non-null   object
 1   WEEKOFMONTH           2999 non-null   int64 
 2   DAYOFWEEK             2999 non-null   object
 3   MAKE                  2999 non-null   object
 4   ACCIDENTAREA          2999 non-null   object
 5   DAYOFWEEKCLAIMED      2999 non-null   object
 6   MONTHCLAIMED          2999 non-null   object
 7   WEEKOFMONTHCLAIMED    2999 non-null   int64 
 8   SEX                   2999 non-null   object
 9   MARITALSTATUS         2999 non-null   object
 10  AGE                   2999 non-null   int64 
 11  FAULT                 2999 non-null   object
 12  POLICYTYPE            2999 non-null   object
 13  VEHICLECATEGORY       2999 non-null   object
 14  VEHICLEPRICE          2999 non-null   object
 15  REPNUMBER             2999 non-null   

In [14]:
# OneHotEncoding on Train (fit & transform)
# OneHotEncoding is to be done on Categorical variables.
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
Xcat = pd.DataFrame(ohe.fit_transform(Xtrain[categoricalFeatures]),columns=ohe.get_feature_names(),index=Xtrain.index)
Xtrain = pd.concat([Xtrain,Xcat],axis=1)
Xtrain.drop(labels=categoricalFeatures,axis=1,inplace=True)
Xtrain.sample(5)

,WEEKOFMONTH,WEEKOFMONTHCLAIMED,AGE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,YEAR,x0_Apr,x0_Aug,x0_Dec,...,x21_4_to_8_years,x21_no_change,x21_under_6_months,x22_1-vehicle,x22_2-vehicles,x22_3_to_4,x22_5_to_8,x23_All_Perils,x23_Collision,x23_Liability
1357,1,1,23,14,400,2,1995,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2634,3,3,32,1,400,1,1996,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
514,3,4,29,8,400,3,1994,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2608,2,2,55,1,400,3,1996,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1894,3,3,0,7,400,4,1995,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# OneHotEncoding on Test (only transform)
# OneHotEncoding is to be done on Categorical variables.
Xcat = pd.DataFrame(ohe.transform(Xtest[categoricalFeatures]),columns=ohe.get_feature_names(),index=Xtest.index)
Xtest = pd.concat([Xtest,Xcat],axis=1)
Xtest.drop(labels=categoricalFeatures,axis=1,inplace=True)
Xtest.sample(5)

,WEEKOFMONTH,WEEKOFMONTHCLAIMED,AGE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,YEAR,x0_Apr,x0_Aug,x0_Dec,...,x21_4_to_8_years,x21_no_change,x21_under_6_months,x22_1-vehicle,x22_2-vehicles,x22_3_to_4,x22_5_to_8,x23_All_Perils,x23_Collision,x23_Liability
5284,5,5,26,3,400,3,1994,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
10645,2,2,42,11,400,1,1995,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
11929,3,3,28,6,400,3,1994,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
11074,4,2,38,4,400,2,1994,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
7495,5,2,33,2,400,1,1995,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
dt = DecisionTreeClassifier()
dt.fit(Xtrain, Ytrain)

DecisionTreeClassifier()

In [17]:
rf = RandomForestClassifier()
rf.fit(Xtrain, Ytrain)

RandomForestClassifier()

In [18]:
X_Pred = dt.predict(Xtest)
XPred = dt.predict(Xtrain)
#Model Accuracy
print("Train Accuracy:", metrics.accuracy_score(Ytrain,XPred))
print("Test Accuracy:", metrics.accuracy_score(Ytest,X_Pred))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,X_Pred))
print("Max Depth",dt.get_depth())
print("Leaf",dt.get_n_leaves())
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, X_Pred))

Train Accuracy: 1.0
Test Accuracy: 0.8827217835578263
Confusion Matrix for Decision Tree:
[[10959  1461]
 [   54   444]]
Max Depth 22
Leaf 289
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

         0.0       1.00      0.88      0.94     12420
         1.0       0.23      0.89      0.37       498

    accuracy                           0.88     12918
   macro avg       0.61      0.89      0.65     12918
weighted avg       0.97      0.88      0.91     12918



In [19]:
X_Pred1 = rf.predict(Xtest)
XPred1 = rf.predict(Xtrain)
#Model Accuracy
print("Train Accuracy:", metrics.accuracy_score(Ytrain,XPred1))
print("Test Accuracy:", metrics.accuracy_score(Ytest,X_Pred1))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,X_Pred1))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, X_Pred1))

Train Accuracy: 1.0
Test Accuracy: 0.960907261185942
Confusion Matrix for Decision Tree:
[[11992   428]
 [   77   421]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98     12420
         1.0       0.50      0.85      0.63       498

    accuracy                           0.96     12918
   macro avg       0.74      0.91      0.80     12918
weighted avg       0.97      0.96      0.97     12918



In [33]:
#Hyperparameter tuning done for decision tree classifier

#RANDOM SEARCH--------------------------------------------

import time
start_time = time.time()

print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(10,300,10),
            'max_depth': range(5,30,2),
            'criterion':['gini','entropy']}
dt_random = RandomizedSearchCV(dt,parameters,n_iter=25,cv=5)
dt_random.fit(Xtrain, Ytrain)
dt_random.cv_results_
df1=pd.DataFrame(dt_random.cv_results_)
grid_parm=dt_random.best_params_
print(grid_parm)
print("accuracy Score for Decision Tree:{0:6f}".
      format(dt_random.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))

RandomizedSearchCV-Decision tree
{'min_samples_leaf': 30, 'max_depth': 25, 'criterion': 'entropy'}
accuracy Score for Decision Tree:0.897430
--- 6.436131000518799 seconds ---


In [21]:
df1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_leaf,param_max_depth,param_criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.040081,0.008286,0.004227,0.005580,90,7,gini,"{'min_samples_leaf': 90, 'max_depth': 7, 'crit...",0.835000,0.886667,0.880000,0.866667,0.883139,0.870294,0.018900,8
1,0.032863,0.002762,0.004386,0.006126,220,19,gini,"{'min_samples_leaf': 220, 'max_depth': 19, 'cr...",0.866667,0.866667,0.866667,0.866667,0.868114,0.866956,0.000579,10
2,0.027767,0.006117,0.008738,0.006897,290,9,entropy,"{'min_samples_leaf': 290, 'max_depth': 9, 'cri...",0.866667,0.866667,0.866667,0.866667,0.868114,0.866956,0.000579,10
3,0.034087,0.006718,0.005964,0.007318,150,25,gini,"{'min_samples_leaf': 150, 'max_depth': 25, 'cr...",0.866667,0.866667,0.878333,0.866667,0.886477,0.872962,0.008129,5
4,0.027591,0.006441,0.009626,0.007871,290,15,entropy,"{'min_samples_leaf': 290, 'max_depth': 15, 'cr...",0.866667,0.866667,0.866667,0.866667,0.868114,0.866956,0.000579,10
5,0.046116,0.007865,0.004005,0.006578,30,9,entropy,"{'min_samples_leaf': 30, 'max_depth': 9, 'crit...",0.908333,0.898333,0.875000,0.861667,0.886477,0.885962,0.016518,2
6,0.032774,0.004594,0.006266,0.007675,200,9,gini,"{'min_samples_leaf': 200, 'max_depth': 9, 'cri...",0.866667,0.866667,0.866667,0.866667,0.868114,0.866956,0.000579,10
7,0.034991,0.004394,0.000608,0.000814,290,15,gini,"{'min_samples_leaf': 290, 'max_depth': 15, 'cr...",0.866667,0.866667,0.866667,0.866667,0.868114,0.866956,0.000579,10
8,0.035090,0.003167,0.009406,0.007680,90,13,gini,"{'min_samples_leaf': 90, 'max_depth': 13, 'cri...",0.835000,0.886667,0.880000,0.866667,0.883139,0.870294,0.018900,8
9,0.031672,0.008283,0.004136,0.006076,210,5,entropy,"{'min_samples_leaf': 210, 'max_depth': 5, 'cri...",0.866667,0.866667,0.866667,0.866667,0.868114,0.866956,0.000579,10


In [22]:
Second_best = df1.loc[df1['rank_test_score'] == 2]
Second_best

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_leaf,param_max_depth,param_criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,0.046116,0.007865,0.004005,0.006578,30,9,entropy,"{'min_samples_leaf': 30, 'max_depth': 9, 'crit...",0.908333,0.898333,0.875,0.861667,0.886477,0.885962,0.016518,2


In [26]:
#dtRand1 = DecisionTreeClassifier()
#dtRand1.fit(Xtrain,Ytrain)
#dtRand_predict1 = dtRand.predict(Xtest)
#print("Test Accuracy:", metrics.accuracy_score(Ytest,dtRand_predict1))
#pd.DataFrame(df1["params"])

In [24]:
#GRID SEARCH----------------------------------------

import time
start_time = time.time()

print("GridSearchCV-Decision tree")
dt_grid = GridSearchCV(dt,parameters,cv=5)
dt_grid.fit(Xtrain, Ytrain)
grid_parm1=dt_grid.best_params_
#dt_grid.cv_results_
#df2=pd.DataFrame(dt_grid.cv_results_)
print(grid_parm1)
print("accuracy Score for Decision Tree:{0:6f}".
      format(dt_grid.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))

GridSearchCV-Decision tree
{'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 30}
accuracy Score for Decision Tree:0.897430
--- 162.30924534797668 seconds ---


In [35]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
dtRand = DecisionTreeClassifier(**grid_parm)
dtGrid = DecisionTreeClassifier(**grid_parm1)

dtRand.fit(Xtrain,Ytrain)
dtRand_predict = dtRand.predict(Xtest)
dtGrid.fit(Xtrain,Ytrain)
dtGrid_predict = dtGrid.predict(Xtest)

In [36]:
# Accuracy for Decision Tree using Random Search CV for Hyperparameter Tuning

print("Test Accuracy:", metrics.accuracy_score(Ytest,dtRand_predict))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,dtRand_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, dtRand_predict))
clf_cv_score = cross_val_score(dtRand, Xtrain, Ytrain, cv=5, scoring="balanced_accuracy")
print(clf_cv_score)

Test Accuracy: 0.8974299427155906
Confusion Matrix for Decision Tree:
[[11427   993]
 [  332   166]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

         0.0       0.97      0.92      0.95     12420
         1.0       0.14      0.33      0.20       498

    accuracy                           0.90     12918
   macro avg       0.56      0.63      0.57     12918
weighted avg       0.94      0.90      0.92     12918

[0.74615385 0.69807692 0.54182692 0.50240385 0.56962025]


In [37]:
#Hyperparameter tuning done for random forest classifier

#RANDOM SEARCH--------------------------------------------

import time
start_time = time.time()

print("RandomizedSearchCV-Random forest")
rand_parameters={'min_samples_leaf' : range(10,100,10),
                 'max_depth': range(1,10,2),
                 'max_features':[10,20,30],
                 'n_estimators':[20,30,40]}
rf_random = RandomizedSearchCV(rf,rand_parameters,n_iter=25,cv=5)
rf_random.fit(Xtrain, Ytrain)
grid_parm=rf_random.best_params_
print(grid_parm)
print("accuracy Score for Decision Tree:{0:6f}".
      format(rf_random.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))

RandomizedSearchCV-Random forest
{'n_estimators': 20, 'min_samples_leaf': 10, 'max_features': 20, 'max_depth': 9}
accuracy Score for Decision Tree:0.939077
--- 24.27045512199402 seconds ---


In [38]:
import time
start_time = time.time()

print("GridSearchCV-Random Forest")
rf_grid = GridSearchCV(rf,rand_parameters)
rf_grid.fit(Xtrain, Ytrain)
grid_parm1=rf_grid.best_params_
print(grid_parm1)
print("accuracy Score for Decision Tree:{0:6f}".
      format(rf_grid.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))

GridSearchCV-Random Forest
{'max_depth': 9, 'max_features': 30, 'min_samples_leaf': 10, 'n_estimators': 30}
accuracy Score for Decision Tree:0.926227
--- 298.3877639770508 seconds ---


In [39]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier 
rfRand = RandomForestClassifier(**grid_parm)
rfGrid = RandomForestClassifier(**grid_parm1)

rfRand.fit(Xtrain,Ytrain)
rfRand_predict = rfRand.predict(Xtest)
rfGrid.fit(Xtrain,Ytrain)
rfGrid_predict = rfGrid.predict(Xtest)

In [40]:
# Accuracy for Random Forest using Random Search CV for Hyperparameter Tuning

print("Test Accuracy:", metrics.accuracy_score(Ytest,rfRand_predict))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,rfRand_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, rfRand_predict))
clf_cv_score = cross_val_score(rfRand, Xtrain, Ytrain, cv=5, scoring="balanced_accuracy")
print(clf_cv_score)

Test Accuracy: 0.9394643133612014
Confusion Matrix for Decision Tree:
[[12024   396]
 [  386   112]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97     12420
         1.0       0.22      0.22      0.22       498

    accuracy                           0.94     12918
   macro avg       0.59      0.60      0.60     12918
weighted avg       0.94      0.94      0.94     12918

[0.70528846 0.71009615 0.5375     0.5        0.56329114]


In [41]:
# Accuracy for Random Forest using Grid Search for Hyperparameter Tuning

print("Test Accuracy:", metrics.accuracy_score(Ytest,rfGrid_predict))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,rfGrid_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, rfGrid_predict))

Test Accuracy: 0.9316457655983899
Confusion Matrix for Decision Tree:
[[11908   512]
 [  371   127]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.96     12420
         1.0       0.20      0.26      0.22       498

    accuracy                           0.93     12918
   macro avg       0.58      0.61      0.59     12918
weighted avg       0.94      0.93      0.94     12918

